In [1]:
from cards import CardDeck
from hand_rankings import getHandRanking, getPlayerHand, getPlayerHandRanking, hand_ranking_classes
import hand_rankings
import players
from players import Player
import hand_prob
from random import shuffle

In [2]:
class PokerGame():
    
        def __init__(self,num_players=5,rounds=10):
            self.rounds = rounds
            self.players = self.createPlayers(num_players)

            self.num_wins = [0]*len(self.players)

            self.playGame(rounds)

            self.resetGame()

            pass

        def resetGame(self):
            self.players[0].resetPlayers()
            pass

        def playGame(self,rounds):
            rounds = self.rounds
            for round in range(rounds):
                current_players = []
                for player in self.players:
                    if not player.is_bankrupt:
                        current_players.append(player)

                new_round = PokerRound(current_players)
                winner = new_round.getWinner()
                self.changePlayerOrder()

        def changePlayerOrder(self):
            first_player = self.players[:1]
            other_players = self.players[1:]
            self.players = other_players + first_player
            return self.players

        def listWins(self):
            for player in self.players:
                print(player.getNumWins(), player.getCash())
                
        def createPlayers(self, n=5):
            player_list = []
            for i in range(n):
                new_player = Player()
                player_list.append(new_player)
            return player_list
        
        def updateNumWins(self, player_index):
            self.num_wins[player_index] += 1
            pass 
        
        def getPlayers(self):
            return self.players
        
        def getNumWins(self):
            self.resetGame()
            return self.num_wins


class PokerRound(PokerGame):

    def __init__(self, players):

        self.players = players

        if len(self.players) >= 2:

            self.cash_pool = 0

            deck = self.createDeck()

            self.dealToPlayers(self.players, deck)

            self.blind(self.players[0], 3)
            self.blind(self.players[1], 5)
            #self.makeBets(self.players[2:])

            self.dealToCommunity(deck)
            #self.makeBets(self.players)

            self.updateCommunity(deck)
            #self.makeBets(self.players)
            
            self.updateCommunity(deck)
            self.makeBets(self.players)

            self.getPlayerRankings(self.players, deck)

            self.winner = self.determineWinner(self.players, deck)

        else:
            if len(self.players) == 1:
                self.winner = self.players[0]
            else:
                self.winner = "Undetermined"

    def createDeck(self):
        return CardDeck()
    
    def dealToPlayers(self, players, deck):
        for player in players:
            deck.dealToPlayer(player)
        pass

    def dealToCommunity(self, deck):
        deck.dealToCommunity(3)
        self.playerAccessToCommunityCards(self.players, deck)
        pass

    def updateCommunity(self, deck):
        deck.dealToCommunity()
        self.playerAccessToCommunityCards(self.players, deck)
        pass

    def getCommunityCards(self, deck):
        return deck.getCommunityCards()

    def playerAccessToCommunityCards(self, players, deck):
        for player in players:
            community_cards = self.getCommunityCards(deck)
            player.getCommunityCards(community_cards)
        pass

    def blind(self, player, n=0):
        if player.cash >= n:
            player.betBlind(n)
            self.addToCashPool(n)
        else:
            #TODO: Remove player from the game
            pass
        pass

    def makeBets(self, players):
        self.raise_cash = self.cash_pool
        for player in players:
            player.getPlay(self.cash_pool, self.raise_cash)
            if player.do_I_play:
                player_bet = player.raise_val
                if player_bet > self.raise_cash:
                    self.raise_cash = player_bet
                self.addToCashPool(player_bet)
        pass

    def addToCashPool(self, num):
        self.cash_pool += num
        pass

    def getPlayerRankings(self, players, deck):
        player_rankings = []
        for player in players:
            player_rankings.append(getPlayerHandRanking(player, deck))
        return player_rankings

    def determineWinner(self, players, deck):
        current_highest = hand_rankings.HighCard
        current_index = hand_ranking_classes.index(current_highest)
        winner = players[0]

        for player in players:
            player_rank = getPlayerHandRanking(player, deck)
            player_index = hand_ranking_classes.index(player_rank)
            if player_index < current_index:
                current_index = player_index
                winner = player

        winner.addWin()
        winner.addCash(self.cash_pool)

        return winner

    def getWinner(self):
        return self.winner


In [3]:
new_game = PokerGame(rounds=10)
new_game.listWins()

2 100
1 80
3 92
1 92
3 136


In [4]:
#Player strategies
#Hand probabilities
#Bluffing chance
#Chance hand will be higher than others
#Creedence of other players - probability others are bluffing